In [76]:
import pandas as pd
import numpy as np
import itertools
import math
from IPython.display import display
from cmath import isnan, nan
from numpy import dot
from numpy.linalg import norm

from turtle import color
from unicodedata import name
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px

#### Получить исходные данные

In [77]:
pd.set_option('display.max_columns', None)
df0 = pd.read_csv('data.csv', delimiter=',', encoding="utf8")
df = df0.copy(deep=True)

#### Получить характерные для семейств запахи

In [78]:
smellLike = pd.read_csv('smells.csv', delimiter=',', encoding="utf8")

smellLike["семейство"] = smellLike["семейство"].map(lambda elem: str(elem).lower())
smellLike["ассоциации"] = smellLike["ассоциации"].map(lambda elem: str(elem).lower().split(';')) + \
                          smellLike["характеристики"].map(lambda elem: str(elem).lower().split(';'))

itemAss = list(set(itertools.chain.from_iterable(smellLike["ассоциации"].values)))
itemAss.remove('nan')
itemAss.sort()

for item in itemAss:
    smellLike[item] = smellLike["ассоциации"].map(lambda elem: 1 if item in elem else 0)
del smellLike["ассоциации"]
del smellLike["характеристики"]
smellLike


,семейство,активный,амбра,апельсин,белый чай,бергамот,благовоние,бодрящий,ваниль,виноград,гвоздика,герань,горный воздух,грейпфрут,древесный,дым,жизнерадостный,зеленое яблоко,зеленый чай,земля,игривый,имбирь,интенсивный,карамель,кедр,корица,кофе,куркума,лаванда,ладан,лайм,легкий,лимон,листья,массивный,металл,морозная свежесть,морской,мох,мускус,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роза,роскошный,сандал,свежий,сладкий,сливочный,смола,согревающий,спокойный,спортивный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,восточные,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,древесные,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,цитрусовые,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
3,фруктовые,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,свежие,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,шипровые,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,пряные,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,цветочные,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,фужерные,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
9,мускусные,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


#### Преобразовать параметры (пол, шлейф, концентрация, сегмент)

In [79]:
sexDict = { "м": 0, "у": 0.5, "ж": 1 }
df["пол"] = df["пол"].map(lambda elem: sexDict[elem])

In [80]:
tailDict = { "крайне незаметный": 0, "незаметный": 0.25, "средний": 0.5, "заметный": 0.75, "очень заметный": 1}
df["шлейф"] = df["шлейф"].map(lambda elem: tailDict[elem])

In [81]:
conDict = {"освежающая вода": 0, "одеколон": 0.2, "туалетная вода": 0.4, "парфюмерная вода": 0.6, "духи": 0.8, "масляные духи": 1}
df["концентрация"] = df["концентрация"].map(lambda elem: conDict[elem])

In [82]:
segmentDict = {"масс-маркет": 0, "люкс": 0.5, "нишевая парфюмерия": 1}
df["сегмент"] = df["сегмент"].map(lambda elem: segmentDict[elem])

#### Задать матрицу смежности для парфюмерных школ

In [45]:
countryDict = {"Франция": 0, "Италия": 1, "ОАЭ": 2, "США": 3, "Россия": 4, "Германия": 5}

countryMatr = np.zeros((len(countryDict), len(countryDict)))
countryMatr[countryDict["Франция"]][countryDict["Италия"]] = countryMatr[countryDict["Италия"]][countryDict["Франция"]] = 0.5
countryMatr[countryDict["Франция"]][countryDict["ОАЭ"]] = countryMatr[countryDict["ОАЭ"]][countryDict["Франция"]] = 0.9
countryMatr[countryDict["Франция"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["Франция"]] = 0.9
countryMatr[countryDict["Франция"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["Франция"]] = 0.8
countryMatr[countryDict["Франция"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Франция"]] = 1

countryMatr[countryDict["Италия"]][countryDict["ОАЭ"]] = countryMatr[countryDict["ОАЭ"]][countryDict["Италия"]] = 0.8
countryMatr[countryDict["Италия"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["Италия"]] = 0.5
countryMatr[countryDict["Италия"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["Италия"]] = 0.9
countryMatr[countryDict["Италия"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Италия"]] = 1

countryMatr[countryDict["ОАЭ"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["ОАЭ"]] = 0.9
countryMatr[countryDict["ОАЭ"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["ОАЭ"]] = 0.8
countryMatr[countryDict["ОАЭ"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["ОАЭ"]] = 1

countryMatr[countryDict["США"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["США"]] = 0.7
countryMatr[countryDict["США"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["США"]] = 1

countryMatr[countryDict["Россия"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Россия"]] = 1

#### Преобразование оставшихся данных

In [46]:
df["верхние ноты"] = df["верхние ноты"].map(lambda elem: str(elem).lower().split(';'))
df["ноты сердца"] = df["ноты сердца"].map(lambda elem: str(elem).lower().split(';'))
df["базовые ноты"] = df["базовые ноты"].map(lambda elem: str(elem).lower().split(';'))
df["семейства"] = df["семейства"].map(lambda elem: str(elem).lower().split(';'))
df["стойкость (ч)"] = df["стойкость (ч)"].values / max(df["стойкость (ч)"].values)

In [47]:
df["цена за 1 мл"] = df["цена"] / df["мл"]
df["цена за 1 мл"] = (df["цена за 1 мл"].values - min(df["цена за 1 мл"].values)) / (max(df["цена за 1 мл"].values) - min(df["цена за 1 мл"].values))

In [48]:
itemSmells = list(set(itertools.chain.from_iterable(df["верхние ноты"].values)) | 
             set(itertools.chain.from_iterable(df["ноты сердца"].values))  |
             set(itertools.chain.from_iterable(df["базовые ноты"].values)))
itemSmells.sort()
for item in itemSmells:
    df[item] = df["верхние ноты"].map(lambda elem: 1 if item in elem else 0) | \
               df["ноты сердца"].map(lambda elem: 1 if item in elem else 0) | \
               df["базовые ноты"].map(lambda elem: 1 if item in elem else 0)

itemFamily = list(set(itertools.chain.from_iterable(df["семейства"].values)))
for item in itemFamily:
    df[item + " семейства"] = df["семейства"].map(lambda elem: 1 if item in elem else 0)

for smell in smellLike.keys().drop('семейство'):
    df[smell] = df["семейства"].map(lambda elem:
        1 if sum(row[smell] for row in smellLike.iloc if row['семейство'] in elem) else 0)

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/62/g379k2017lgcvk7cq8k6vsbw0000gn/T/ipykernel_90924/3757279040.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

#### Некоторые параметры

In [49]:
F_NAME = "название"
F_DIST = "расстояние"

nameArr = df["название"]

dfTree = pd.DataFrame(columns=["название", "семейства"], data=df[["название", "семейства"]].values)

In [50]:
del df["верхние ноты"]
del df["ноты сердца"]
del df["базовые ноты"]
del df["семейства"]
del df["мл"]
del df["цена"]

In [51]:
display(df)

,название,пол,бренд,страна,сегмент,концентрация,стойкость (ч),шлейф,цена за 1 мл,альдегиды,амбра,амбра,амброксан,ананас,апельсин,апельсиновый цвет,арбуз,белые цветы,белый перец,белый персик,бергамот,береза,бобы,ваниль,ваниль,ветивер,ветивер,вишня,гардения,гвоздика,герань,грейпфрут,груша,древесный аккорд,дыня,жасмин,жасмин,замша,зеленый аккорд,злаки,иланг-иланг,имбирь,ирис,какао,карамель,кардамон,кашмеран,кедр,киви,клубника,кожа,кожаный аккорд,корень фиалки,кориандр,корица,коричневый сахар,кофе,красное яблоко,лабданум,лаванда,лавр,ладан,лакричник,ландыш,лен,ликёр,лилия,лимон,личи,магнолия,малина,мандарин,маракуйя,миндаль,можжевельник,мох,мускатный орех,мускус,мускус,нарцисс,нероли,опопанакс,орхидея,османтус,палисандр,пачули,перуанский бальзам,пион,полынь,ревень,роза,розмарин,розовый перец,сандал,силколид,сирень,сладкая вата,слива,танжерин,тимьян,тмин,тубероза,фиалка,франжипани,цвет персика,цибетин,цикламен,цитрусы,чай,черная смородина,черника,черный перец,шалфей,шафран,элеми,яблоко,янтарь,цветочные семейства,цитрусовые семейства,шипровые семейства,ванильные семейства,фруктовые семейства,сладкие семейства,древесные семейства,восточные семейства,водные семейства,пряные семейства,ароматические семейства,фужерные семейства,активный,белый чай,благовоние,бодрящий,виноград,горный воздух,древесный,дым,жизнерадостный,зеленое яблоко,зеленый чай,земля,игривый,интенсивный,куркума,лайм,легкий,листья,массивный,металл,морозная свежесть,морской,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роскошный,свежий,сладкий,сливочный,смола,согревающий,спокойный,спортивный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,Shalimar,1.0,Guerlain,Франция,0.5,0.8,0.069444,0.50,0.302066,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,3 L'Imperatrice,1.0,Dolce&Gabbana,Италия,0.5,0.4,0.138889,1.00,0.167144,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Resolute Gold,0.0,Khalis,ОАЭ,0.0,0.6,0.222222,0.50,0.000000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Oath for Him,0.0,Ajmal,ОАЭ,0.5,0.6,0.041667,0.50,0.208645,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,Twist,1.0,Emper,ОАЭ,0.0,0.6,0.166667,0.75,0.000830,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,CK Everyone,0.5,Calvin Klein,США,0.5,0.4,0.06944

In [52]:
del df["название"]
display(df)

,пол,бренд,страна,сегмент,концентрация,стойкость (ч),шлейф,цена за 1 мл,альдегиды,амбра,амбра,амброксан,ананас,апельсин,апельсиновый цвет,арбуз,белые цветы,белый перец,белый персик,бергамот,береза,бобы,ваниль,ваниль,ветивер,ветивер,вишня,гардения,гвоздика,герань,грейпфрут,груша,древесный аккорд,дыня,жасмин,жасмин,замша,зеленый аккорд,злаки,иланг-иланг,имбирь,ирис,какао,карамель,кардамон,кашмеран,кедр,киви,клубника,кожа,кожаный аккорд,корень фиалки,кориандр,корица,коричневый сахар,кофе,красное яблоко,лабданум,лаванда,лавр,ладан,лакричник,ландыш,лен,ликёр,лилия,лимон,личи,магнолия,малина,мандарин,маракуйя,миндаль,можжевельник,мох,мускатный орех,мускус,мускус,нарцисс,нероли,опопанакс,орхидея,османтус,палисандр,пачули,перуанский бальзам,пион,полынь,ревень,роза,розмарин,розовый перец,сандал,силколид,сирень,сладкая вата,слива,танжерин,тимьян,тмин,тубероза,фиалка,франжипани,цвет персика,цибетин,цикламен,цитрусы,чай,черная смородина,черника,черный перец,шалфей,шафран,элеми,яблоко,янтарь,цветочные семейства,цитрусовые семейства,шипровые семейства,ванильные семейства,фруктовые семейства,сладкие семейства,древесные семейства,восточные семейства,водные семейства,пряные семейства,ароматические семейства,фужерные семейства,активный,белый чай,благовоние,бодрящий,виноград,горный воздух,древесный,дым,жизнерадостный,зеленое яблоко,зеленый чай,земля,игривый,интенсивный,куркума,лайм,легкий,листья,массивный,металл,морозная свежесть,морской,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роскошный,свежий,сладкий,сливочный,смола,согревающий,спокойный,спортивный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,1.0,Guerlain,Франция,0.5,0.8,0.069444,0.50,0.302066,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,1.0,Dolce&Gabbana,Италия,0.5,0.4,0.138889,1.00,0.167144,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,Khalis,ОАЭ,0.0,0.6,0.222222,0.50,0.000000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,Ajmal,ОАЭ,0.5,0.6,0.041667,0.50,0.208645,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
4,1.0,Emper,ОАЭ,0.0,0.6,0.166667,0.75,0.000830,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0.5,Calvin Klein,США,0.5,0.4,0.069444,0.50,0.091255,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

#### Матрица смежности для семейств (1 уровень)

In [53]:
layer1 = {"восточные": 0, "древесные": 1, "цветочные": 2, "цитрусовые": 3}

treeLayer1 = np.zeros((len(layer1), len(layer1)))
treeLayer1[layer1["восточные"]][layer1["древесные"]] = treeLayer1[layer1["древесные"]][layer1["восточные"]] = 0.5
treeLayer1[layer1["восточные"]][layer1["цветочные"]] = treeLayer1[layer1["цветочные"]][layer1["восточные"]] = 0.7
treeLayer1[layer1["восточные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["восточные"]] = 0.9
treeLayer1[layer1["древесные"]][layer1["цветочные"]] = treeLayer1[layer1["цветочные"]][layer1["древесные"]] = 0.7
treeLayer1[layer1["древесные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["древесные"]] = 0.9
treeLayer1[layer1["цветочные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["цветочные"]] = 0.7

#### Матрица смежности для семейств (2 уровень)

In [54]:
layer2 = {"цветочные": 0, "древесные": 1, "пряные": 2, "ванильные": 3, "фужерные": 4, "водные": 5, "свежие": 6, "фруктовые": 7, "сладкие": 8, "ароматические": 9}

treeLayer2 = np.zeros((len(layer2), len(layer2)))
treeLayer2[layer2["цветочные"]][layer2["древесные"]] = treeLayer2[layer2["древесные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["пряные"]] = treeLayer2[layer2["пряные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["цветочные"]] = 0.6
treeLayer2[layer2["цветочные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["цветочные"]] = 0.2
treeLayer2[layer2["цветочные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["цветочные"]] = 0.3

treeLayer2[layer2["древесные"]][layer2["пряные"]] = treeLayer2[layer2["пряные"]][layer2["древесные"]] = 0.4
treeLayer2[layer2["древесные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["древесные"]] = 0.5
treeLayer2[layer2["древесные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["древесные"]] = 0.3
treeLayer2[layer2["древесные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["древесные"]] = 0.6
treeLayer2[layer2["древесные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["древесные"]] = 0.5
treeLayer2[layer2["древесные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["древесные"]] = 0.6
treeLayer2[layer2["древесные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["древесные"]] = 0.4
treeLayer2[layer2["древесные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["древесные"]] = 0.4

treeLayer2[layer2["пряные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["пряные"]] = 0.5
treeLayer2[layer2["пряные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["пряные"]] = 0.3
treeLayer2[layer2["пряные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["пряные"]] = 0.3
treeLayer2[layer2["пряные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["пряные"]] = 0.3

treeLayer2[layer2["ванильные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["ванильные"]] = 0.5
treeLayer2[layer2["ванильные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["ванильные"]] = 0.7
treeLayer2[layer2["ванильные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["ванильные"]] = 0.6
treeLayer2[layer2["ванильные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["ванильные"]] = 0.4
treeLayer2[layer2["ванильные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["ванильные"]] = 0.3
treeLayer2[layer2["ванильные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["ванильные"]] = 0.5

treeLayer2[layer2["фужерные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["фужерные"]] = 0.5
treeLayer2[layer2["фужерные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["фужерные"]] = 0.5

treeLayer2[layer2["водные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["водные"]] = 0.3
treeLayer2[layer2["водные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["водные"]] = 0.7
treeLayer2[layer2["водные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["водные"]] = 0.6
treeLayer2[layer2["водные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["водные"]] = 0.6

treeLayer2[layer2["свежие"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["свежие"]] = 0.5
treeLayer2[layer2["свежие"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["свежие"]] = 0.6
treeLayer2[layer2["свежие"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["свежие"]] = 0.3

treeLayer2[layer2["фруктовые"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["фруктовые"]] = 0.3
treeLayer2[layer2["фруктовые"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["фруктовые"]] = 0.3

treeLayer2[layer2["сладкие"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["сладкие"]] = 0.6

#### Матрица смежности для семейств (3 уровень)

In [55]:
layer3 = {"свежие": 0, "сладкие": 1, "мускусные": 2, "шипровые": 3, "амбровые": 4, "фруктовые": 5, "-": 6}

treeLayer3 = np.zeros((len(layer3), len(layer3)))
treeLayer3[layer3["свежие"]][layer3["сладкие"]] = treeLayer3[layer3["сладкие"]][layer3["свежие"]] = 0.6
treeLayer3[layer3["свежие"]][layer3["мускусные"]] = treeLayer3[layer3["мускусные"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["свежие"]] = 0.6
treeLayer3[layer3["свежие"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["свежие"]] = 1

treeLayer3[layer3["сладкие"]][layer3["мускусные"]] = treeLayer3[layer3["мускусные"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["сладкие"]] = 1

treeLayer3[layer3["мускусные"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["мускусные"]] = 0.2
treeLayer3[layer3["мускусные"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["мускусные"]] = 0.6
treeLayer3[layer3["мускусные"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["мускусные"]] = 0.6
treeLayer3[layer3["мускусные"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["мускусные"]] = 1

treeLayer3[layer3["шипровые"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["шипровые"]] = 0.6
treeLayer3[layer3["шипровые"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["шипровые"]] = 0.6
treeLayer3[layer3["шипровые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["шипровые"]] = 1

treeLayer3[layer3["амбровые"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["амбровые"]] = 0.7
treeLayer3[layer3["амбровые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["амбровые"]] = 1

treeLayer3[layer3["фруктовые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["фруктовые"]] = 1

In [56]:
layer = [layer1, layer2, layer3]
tree = [treeLayer1, treeLayer2, treeLayer3]

### Данные

In [57]:
excludeFields = ["цена за 1 мл", "пол", "бренд", 
                 "страна", "сегмент", "концентрация"]

def getDataFrameAroma(df):
    dfNew = df.copy()
    for elem in excludeFields:
        del dfNew[elem]
    
    return dfNew

def getDataFrameStat(df):
    dfStatParams = pd.DataFrame(columns=excludeFields, data=df[excludeFields].values)
    del dfStatParams["бренд"]
    del dfStatParams["страна"]

    return dfStatParams

#### Получение расстояний

In [58]:
def _complete(v1, v2):
    for i in range(len(v1) - len(v2)):
        v2.append("-")
    return v2

def complete(v1, v2):
    n1, n2 = len(v1), len(v2)
    if n1 > n2:
        v2 = _complete(v1, v2)
    elif n1 < n2:
        v1 = _complete(v2, v1)
    return v1, v2

# Расстояния
def getDistance(v1, v2, nPow):
    res = 0
    for i in range(len(v1)):
        if isnan(v1[i]) or isnan(v2[i]):
            continue
        res += pow(abs(v1[i] - v2[i]), nPow)
    return pow(res, 1 / nPow)

# Манхэттенское расстояние
def getManhattanDistance(v1, v2):
    return getDistance(v1, v2, 1)

# Евклидово расстояние
def getEuclideanDistance(v1, v2):
    return getDistance(v1, v2, 2)

# Косинусное 
def getCos(v1, v2):
    v1T, v2T = v1.copy(), v2.copy()
    n = len(v1)
    indArr = [i for i, (elem1, elem2) in enumerate(zip(v1T, v2T)) if isnan(elem1) or isnan(elem2)]
    v1T[:] = [elem for i, elem in enumerate(v1T) if i not in indArr]
    v2T[:] = [elem for i, elem in enumerate(v2T) if i not in indArr]
    return 1 - dot(v1T, v2T) / (norm(v1T) * norm(v2T))

# Расстояние по дереву
def getTreeDistance(v1, v2):
    res = 0
    resArr = []
    v1T, v2T = v1, v2
    if len(v1) != len(v2):
        v1T, v2T = complete(v1, v2)
    for i in range(len(v1T)):
        res += tree[i][layer[i][v1T[i]]][layer[i][v2T[i]]]
    
    return res / len(tree)

# Сравнение брендов
def getBrandDistance(v1, v2):
    return 0 if v1[0] == v2[0] else 1

# Сравнение стран
def getCountryDistance(v1, v2):
    return countryMatr[countryDict[v1]][countryDict[v2]]

# Найти все похожие
def getSimilarity(id, matr, nameArr):
    data = matr[id]
    res = pd.DataFrame(zip(data, nameArr), index=np.arange(len(matr)), columns=["расстояние", "название"])
    return res.sort_values("расстояние")

In [59]:
# Мера Жаккара
def _getJacquard(v1, v2):
    a = v1.count(1)
    b = v2.count(1)
    c = 0
    for i in range(len(v1)):
        if v1[i] and v2[i]:
            c += 1
    return 1 - c / (a + b - c)


def getJacquard(df, nameArr):
    matrData = df.values.tolist()
    n = len(matrData)
    matrRes = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrRes[i][j] = matrRes[j][i] = _getJacquard(matrData[i], matrData[j])
    return matrRes

In [60]:
def calcDistance(f, df):
    matrData = df.values.tolist()
    n = len(matrData)
    matrRes = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrRes[i][j] = matrRes[j][i] = f(matrData[i], matrData[j])
    return matrRes / matrRes.max()   


def calcDistanceCompined(df, dfTree):
    dfTree = dfTree["семейства"]
    dfMan = getDataFrameAroma(df)
    dfJac = dfMan.copy()
    del dfJac["шлейф"]

    dfStatParams = getDataFrameStat(df)
    
    matrTree = calcDistance(getTreeDistance, dfTree)
    # matrCos = calcDistance(getCos, dfMan)
    matrEucl = calcDistance(getEuclideanDistance, dfStatParams)
    # matrMan = calcDistance(getManhattanDistance, dfMan)
    matrBrand = calcDistance(getBrandDistance, df["бренд"])
    matrCountry = calcDistance(getCountryDistance, df["страна"])
    matrJac = getJacquard(dfJac, nameArr)

    xTree = matrTree.max()
    # xCos = matrCos.max()
    xEuci = matrEucl.max()
    # xMan = matrMan.max()
    xJac = matrJac.max()
    xBrand = matrBrand.max()
    xCountry = matrCountry.max()


    kJac, kTree, kEuci, kBrand, kCountry = 2, 0.5, 10, 2, 2

    return (kJac * matrJac + kTree * matrTree + matrEucl + kBrand * matrBrand + kCountry * matrCountry) / (kJac * xJac + kTree * xTree + xEuci + kBrand * xBrand + kCountry * xCountry)

#### Графическое изображение

In [61]:
def draw(matrRes, nameArr, title, color='Inferno'):
    fig = px.imshow(matrRes, x=nameArr, y=nameArr, color_continuous_scale=color, title=title)
    fig.update_layout(width=1000, height=1200)
    fig.update_traces(text=nameArr)
    fig.update_xaxes(side="top")
    fig.show()

### Демонстрация расстояний

#### Манхэттенское расстояние

In [62]:
matrRes = calcDistance(getManhattanDistance, getDataFrameAroma(df))
draw(matrRes, nameArr, "Манхэттенское расстояние")
# getDifference(11, matrRes, nameArr)

#### Евклидово расстояние

In [63]:
matrRes = calcDistance(getEuclideanDistance, getDataFrameStat(df))
draw(matrRes, nameArr, "Евклидово расстояние")

#### Расстояние по брендам

In [64]:
matrRes = calcDistance(getBrandDistance, df["бренд"])
draw(matrRes, nameArr, "Расстояние по брендам")

#### Расстояние по парфюмерным школам

In [65]:
matrRes = calcDistance(getCountryDistance, df["страна"])
draw(matrRes, nameArr, "Расстояние по парфюмерным школам")

#### Косинусное сходство

In [66]:
matrRes = calcDistance(getCos, getDataFrameAroma(df))
draw(matrRes, nameArr, "Косинусное подобие")

#### Мера Жаккара

In [67]:
tempDF = getDataFrameAroma(df)
del tempDF["шлейф"]

matrRes = getJacquard(tempDF, nameArr)
draw(matrRes, nameArr, "Мера Жаккара")

#### Расстояние по дереву

In [68]:
matrRes = calcDistance(getTreeDistance, dfTree["семейства"])
draw(matrRes, nameArr, "Расстояние по дереву")


#### Комбинированная мера

In [69]:
draw(calcDistanceCompined(df, dfTree), nameArr, "Комбинированная мера")

#### Требуемые параметры

In [70]:
matrSimilarity = calcDistanceCompined(df, dfTree)

### Задача 1

**Вход**: 1 объект (затравочный). 

**Выход**: список рекомендаций, ранжированный по убыванию близости с затравкой. Примените Вашу обобщающую меру близости.

In [71]:
def printRes(arr):
    print("расстояние \t\t\t название")
    for elem in arr:
        for key, value in elem.items():
            print("{0}\t\t{1}".format(value, key))

In [35]:
def _findSimilar(name):
    ind = df0[F_NAME].tolist().index(name)
    
    listSimilarity = getSimilarity(ind, matrSimilarity, nameArr)
    return listSimilarity


def findSimilar(name):
    listSimilarity = _findSimilar(name)

    return listSimilarity[listSimilarity[F_NAME] != name]
    
res = findSimilar("Lost Cherry")
print(res)

    расстояние                                   название
24    0.327207                           Fucking Fabulous
21    0.489062                                  Bombshell
5     0.598768                                CK Everyone
6     0.628199                                    Bad Boy
16    0.629813           Flora by Gucci Gorgeous Gardenia
1     0.699999                            3 L'Imperatrice
17    0.703969      La Petite Robe Noire Ma Robe Cocktail
19    0.712539                           Fleur Narcotique
18    0.743803  La Petite Robe Noire Ma Robe Sous le Vent
15    0.750129                               The Only One
11    0.758375                      Lanvin Eclat D Arpege
25    0.773888                                   Invictus
12    0.782934                           MOSCHINO Toy Boy
14    0.810475                                 Channel №5
4     0.827145                                      Twist
2     0.841354                              Resolute Gold
8     0.857192

### Задача 2

**Вход**: массив объектов (лайков). 

**Выход**: сформированный ранжированный список рекомендаций.

In [36]:
from collections import defaultdict


def _findSimilarMany(nameArr):
    recList = []
    for name in nameArr:
        rec = _findSimilar(name)
        recList.append(rec.loc[rec[F_NAME].isin(nameArr) == False])

    dfRes = defaultdict(lambda: 1e2)
    for rec in recList:
        for i, row in rec.iterrows():
            curName = row[F_NAME]
            curDist = row[F_DIST]
            dfRes[curName] = min(dfRes[curName], curDist)

    return dfRes


def findSimilarMany(nameArr):
    resDict = _findSimilarMany(nameArr)
    return sorted([{key: elem} for key, elem in resDict.items()], key=lambda elem: list(elem.values())[0])

res = findSimilarMany(["Lost Cherry", "Sauvage"])
printRes(res)

расстояние 			 название
0.3272066516370411		Fucking Fabulous
0.37498623945781484		The Only One
0.43470772252108636		Samsara Eau de Parfum
0.4843175700992853		Eau de Lacoste L.12.12 Blanc
0.4890619398825649		Bombshell
0.5066805391136283		Cartier Declaration D Un Soir
0.5121362332843526		Ange ou Demon
0.5127623844526122		Black Opium
0.522873122179254		Shalimar
0.527273130352477		3 L'Imperatrice
0.5987682056970215		CK Everyone
0.6281991349875353		Bad Boy
0.6291869291028859		Fleur Narcotique
0.6298134745297216		Flora by Gucci Gorgeous Gardenia
0.6435362709157637		MOSCHINO Toy Boy
0.6493637328538978		La Petite Robe Noire Ma Robe Sous le Vent
0.6516309379575775		La Petite Robe Noire Ma Robe Cocktail
0.6524820840791236		Invictus
0.6540230266070779		Lanvin Eclat D Arpege
0.6561633596262402		Channel №5
0.7835785956974474		Renata
0.7847650805005003		Oath for Him
0.8271446383763958		Twist
0.8413542292426357		Resolute Gold


### Задача 3

**Вход**: Вход: массив затравочных объектов и массив дизлайков.

**Выход**: сформированный ранжированный список рекомендаций.

In [37]:
def delOpposite(dict, nameArr):
    for name in nameArr:
        if name in dict.keys():
            del dict[name]
    
    return dict


def findSimilar(likesArr, dislikesArr):
    likesRec = delOpposite(_findSimilarMany(likesArr), dislikesArr)
    dislikesRec = delOpposite(_findSimilarMany(dislikesArr), likesArr)

    dictRes = {}
    if len(likesArr) == 0:
        for key, elem in dislikesRec.items():
            if elem > 0.7:
                dictRes[key] = elem
        return sorted([{key: elem} for key, elem in dictRes.items()], key=lambda elem: list(elem.values())[0])

    for key in likesRec.keys():
        if likesRec[key] <= dislikesRec[key]:
            dictRes[key] = likesRec[key]
    return sorted([{key: elem} for key, elem in dictRes.items()], key=lambda elem: list(elem.values())[0])


res = findSimilar([],
                  ["Fleur Narcotique"])
printRes(res)


расстояние 			 название
0.7125385770627655		Lost Cherry
0.7381156151959014		The Only One
0.7726249047149317		MOSCHINO Toy Boy
0.788734820979839		Invictus
0.8130086402389629		Resolute Gold
0.853790839792487		CK Everyone
0.8719328376322022		Oath for Him
0.8922065963362749		Renata
0.8934906277559161		Bad Boy
0.9163098067925642		Fucking Fabulous
